In [ ]:
from tools import train_net_shapenet

class args:
    config_file = "configs/shapenet/voxmesh_R50.yaml"
    opts = ""
    num_gpus = 0
    data_dir = 'ShapeNetV1processed'
    copy_data = False
    no_color = True

In [ ]:
cfg = train_net_shapenet.setup(args())
loaders = train_net_shapenet.setup_loaders(cfg)

In [ ]:
import torch
def rotateback(voxels, P):
    voxel_coords = []
    for i, cur_voxels in enumerate(voxels):              
        cur_voxel_coords = loaders["train"].dataset._coordinates(cur_voxels, P[i].inverse())
        voxel_coords.append(cur_voxel_coords) 
    voxel_coords = tuple(voxel_coords)
    
    voxels = []
    for i, cur_voxel_coords in enumerate(voxel_coords):
        cur_voxels = loaders["train"].dataset._voxelize(cur_voxel_coords, P[0])
        voxels.append(cur_voxels)
    voxels = torch.stack(voxels, dim=0)
    
    return voxels

In [ ]:
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

import os
from os.path import exists

for i, batch in enumerate(loaders["train"]):
    _, _, _, _, _, Ps, id_strs = batch 
    batch = loaders["train"].postprocess(batch, 'cpu')
    _, _, _, _, voxels_gt = batch
    voxels_gt = voxels_gt.float()
    #voxels_gt = rotateback(voxels_gt.cpu(), Ps.cpu())
    #print(voxels_gt.shape)
    
    #words = id_strs[0].split('-')
    #output_dir = "/home/alan/Desktop/meshrcnn_s/meshrcnn_s/datasets/shapenet/predictedVoxel"
    #output_dir = os.path.join(output_dir, words[0], words[1], "predictedVoxels.pt")
    #if not exists(output_dir):
    #    missed.append(output_dir)
    #    continue
    
    #voxel_scores = (torch.load(output_dir).sigmoid()>0.7).float()
    #print(voxel_scores.shape)    
    #voxel_scores = rotateback(voxel_scores.cpu(), Ps.cpu())    

    fig = make_subplots(
    rows=6, cols=2,
    specs=[[{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}]])    
     
    for index in range(0, 6):
 
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxels_gt[index].cpu().detach().numpy().flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
                ),
            row=index + 1,
            col=1,
        )   

        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxel_scores[index].cpu().detach().numpy().flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
                ),
            row=index + 1,
            col=2,
        )
        
        print(np.histogram(voxels_gt[index].detach().numpy().flatten()))
        print("voxel_scores")
        print(np.histogram(voxel_scores[index].cpu().detach().numpy().flatten()))
        
    fig.update_layout(
        autosize=False,
        width=800,
        height=8000,)
        
    fig.show()
     
    break

In [ ]:
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

checkpoint_path = "/home/alan/Desktop/lstm3dtest/cp.ckpt"
model.load_weights(checkpoint_path)
 
for i, batch in enumerate(loaders["train"]):
    _, _, _, _, _, Ps, id_strs = batch     
    batch = loaders["train"].postprocess(batch, 'cpu')
    imgs, meshes_gt, points_gt, normals_gt, voxels_gt = batch
    #voxels_gt_r = rotateback(voxels_gt.cpu(), Ps.cpu())
    voxels_gt = voxels_gt.float()  
    
    words = id_strs[0].split('-')
    mesh_rcnn_predicted_voxels_dir = "/home/alan/Desktop/meshrcnn_s/meshrcnn_s/datasets/shapenet/predictedVoxel"
    mesh_rcnn_predicted_voxels_dir = os.path.join(mesh_rcnn_predicted_voxels_dir, words[0], words[1], "predictedVoxels.pt")
    mesh_rcnn_predicted_voxels = (torch.load(mesh_rcnn_predicted_voxels_dir).sigmoid()>0.7).float()
    mesh_rcnn_predicted_voxels = rotateback(mesh_rcnn_predicted_voxels.cpu(), Ps.cpu()).float()
    mesh_rcnn_predicted_voxels_input = mesh_rcnn_predicted_voxels[None, :, :, :, :, None]
    print("voxels_gt.shape")
    print(voxels_gt.shape)
    print("mesh_rcnn_predicted_voxels_input.shape")
    print(mesh_rcnn_predicted_voxels_input.shape)
    result = model.predict(mesh_rcnn_predicted_voxels_input.cpu().detach().numpy())
    result = (torch.from_numpy(np.squeeze(result))>0.7).float()
    print("result.shape")
    print(result.shape)
    
    fig = make_subplots(
    rows=6, cols=3,
    specs=[[{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],           
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}]])    

    # y_true = [[0, 1], [0, 0]]
    # y_pred = [[-18.6, 0.51], [2.94, -12.8]]
    # Using default 'auto'/'sum_over_batch_size' reduction type.
    bce = tf.keras.losses.BinaryCrossentropy()
    # print(bce(y_true, y_pred).numpy())    
    
    for index in range(0, 6):
      
        voxels_gt_np = voxels_gt[index].cpu().numpy()
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxels_gt_np.flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
                ),
            row=index + 1,
            col=1,
        )   

        print(np.histogram(voxels_gt_np.flatten()))
 
        voxel_probs_np = mesh_rcnn_predicted_voxels[index].cpu().numpy()
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(           
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxel_probs_np.flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
            ),
            row=index + 1,
            col=2,        
        )
        
        print(np.histogram(voxel_probs_np.flatten()))

        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]        
        fig.add_trace( 
            go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=result[index].numpy().flatten(),
            isomin=0,
            isomax=1,
            opacity=0.1, # needs to be small to see through all surfaces
            surface_count=20, # needs to be a large number for good volume rendering
            ),
            row=index + 1,
            col=3,        
        )
        
        print(np.histogram(result[index].flatten()))
    
    fig.show()
    
    #mesh_rcnn_predicted_voxels = rotateback(mesh_rcnn_predicted_voxels.cpu(), Ps.cpu()) 
    bce = tf.keras.losses.BinaryCrossentropy()    
    print("index")
    print(index)
    print(bce(voxels_gt[index].cpu().detach().numpy(), mesh_rcnn_predicted_voxels[index].cpu().detach().numpy()).numpy()) 
    print(bce(voxels_gt[index].cpu().detach().numpy(), result[index]).numpy())

    
    break